한국어 데이터 전처리 및 품질관리  QA or 챗봇
```
  전처리 프로세스
    데이터 수집
      크롤링, 정적인 데이터(csv)
      다양한 표현과 의도를 포함(배송,환불, 불량)
    텍스트 정제
      오타/특수문자 제거 : "ㅋㅋ", "ㅎㅎ" "^^" 이모지 제거
      비속어 처리 : "쓰레기임"->중립표현으로 변경(문제가 있어요) 또는 필터링
      영어/숫자 처리: 도메인에 따라 유지 또는 제거(iPhone은 유지)
    띄어쓰기 교정
      한글은 띄어쓰기 불규칙이 흔함: 배송느림->배송 느림
      도구 pykospacing(띄어쓰기 자동교정), konlpy(형태소분석)
    정규화
      동일한 의미의 표현통일 : 좋음,최고 ->긍정
      답변 일관성 : 2,3일이내 , 이틀에서삼일-> 주문후 2,3일 이내
    토큰화 고려
      한글은 1글자 1토큰, 띄어쓰기 최적화로 토큰 절약
  품질관리
    중복제거
      동일문의/답변제거(set)
    다양성확보      
    수동검증
      데이터의 10% 수동 확인
    통계분석          
      의도분포 : 배송40%, 환불30% ...
```

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-8jga8e9y
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-8jga8e9y
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit b32a889cbd10b006d2f4aba118f0cd5b677e2979
  Preparing metadata (setup.py) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 29.4 MB/s eta 0:00:00


In [ ]:
import openai
import os
import json
from pykospacing import Spacing
import re
from konlpy.tag import Okt

In [ ]:
raw_data = [
    {"question": "배송언제오나요ㅋㅋ", "answer": "2~3일내도착"},
    {"question": "환불어떻게함?", "answer": "7일이내 반품시 환불가능"},
    {"question": "제품이쓰레기임ㅡㅡ", "answer": "고객센터연락주세요"},
    {"question": "배송 언제 오나요?", "answer": "이틀에서 사흘 걸림"},  # 중복 의도
    {"question": "환불안되면어쩌지ㅠㅠ", "answer": "7일안에 반품하면 환불돼요!"}
]

In [ ]:
api_key = 'your_api_key'

In [37]:
client = openai.OpenAI(api_key=api_key)

In [ ]:
# 띄어쓰기 교정기
spacing = Spacing()
# 형태소 분석기
okt = Okt()

In [79]:
# 텍스트 정제 함수
def clean_text(text,is_answer=False):
  # 특수문자 이모지 제거
  if is_answer:
    text =  re.sub(r"[ㅋㅎㅠㅡ-]+|[^가-힣\s()0-9~']","",text)
  else:
    text =  re.sub(r"[ㅋㅎㅠㅡ-]+|[^가-힣\s]","",text)
  # 비속어 필터링(example)  - 비속어 사전을 새로만들거나. 찾아서..
  bad_words = {"쓰레기":"문제가 있는", "게판":"문제가 있는"}
  for bad,good in bad_words.items():
    text = text.replace(bad,good)
  # 연속 공백 제거
  text = re.sub(r"\s+"," ",text)
  return text
# 띄어쓰기 교정 함수
def correct_spacing(text):
  return spacing(text)
# 정규화 함수
def normalize_text(text, is_answer=False):
  text = correct_spacing(clean_text(text,is_answer))
  if is_answer:
    # 답변 존댓말 및 통일
    if "환불"   in text:
      text = "구매 후 7일 이내 반품 요청 시 전액 환불 가능합니다."
    elif "배송" in text or  "도착" in text:
      text = "보통 주문 후 2~3일 내 배송됩니다."
    elif "고객센터" in text:
      text = "고객센터로 연락주세요."
  return text

# 데이터 전처리 함수
def preprocess_data(raw_data):
  processed_data = []
  seen_questions = set()  # 중복 확인용
  for item in raw_data:
    question = normalize_text(item['question'])
    answer = normalize_text(item['answer'], is_answer=True)
    # 중복제거
    if question not in seen_questions:
      processed_data.append({"question": question, "answer": answer})
      seen_questions.add(question)
  return processed_data
# jsonl 파일 생성
def create_jsonl(data, filename):
  # jsonl로 변환
  with open(filename,'w',encoding='utf-8') as f:
    for item in data:
        messages = [
            {"role": "system", "content": "너는 친절한 한글 고객센터 쳇봇이야. 최대한 친절하게 답해줘"},
            {"role": "user", "content": item['question']},
            {"role": "assistant", "content": item['answer']}
        ]
        json.dump({"messages" : messages}, f, ensure_ascii=False)
        f.write('\n')
# 품질관리 : 질문의 의도를 분석
from collections import Counter
def analyze_intents(processed_data):
  intents = []
  for item in processed_data:
    tokens = okt.pos(item['question'], norm=True, stem=True)
    intent = [token[0] for token in tokens if token[1] in ['Noun','Verb']][0]
    intents.append(intent)
  return dict(Counter(intents))

In [134]:
# 1. 데이터 전처리
processed_data = preprocess_data(raw_data)
print(processed_data)
# 2. jsonl 생성
create_jsonl(processed_data, 'processed_data.jsonl')
# 3 품질측정(질문의도)
intents = analyze_intents(processed_data)
print(intents)

[{'question': '배송 언제 오나요', 'answer': '보통 주문 후 2~3일 내 배송됩니다.'}, {'question': '환불 어떻게 하나요', 'answer': '구매 후 7일 이내 반품 요청 시 전액 환불 가능합니다.'}, {'question': '제품이 망가졌어요', 'answer': '구매 후 7일 이내 반품 요청 시 전액 환불 가능합니다.'}, {'question': '이 거 왜 이래요', 'answer': '자세한 상황을 말씀해 주시면 도와 드리겠습니다'}, {'question': '환불 안 해주면 고소할게요', 'answer': '구매 후 7일 이내 반품 요청 시 전액 환불 가능합니다.'}]
{'배송': 1, '환불': 2, '제품': 1, '이': 1}


```
1. 의도기반 설계
  문의유형별 분류 : 배송,환불,불량,할인,일반문의
  각 의도별 데이터쌍..5만개 이상
2. 라벨링 전략
  다양한 표현 : 배송언제,물건언제도착,택배언제와
  존댓말 :고객지원 ~ 입니다(습니다) 통일
  수동 : 직접 생산
  자동 : 네이버 스토어 Q&A -> 크롤링해서 사용
  반자동 : chatgpt이용해서 생성하고 사람이 검증
3. Edge-Case 대응
  비표준적이거나 처리 어려운 표현
    모호문의 : 이거 왜 이래 -> "자세한 상황을 .... 도와드리.."
    민감문의 : 환불 안해주면 고소할꺼야->""
    다중의도: 배송은 언제 그리고 환불은 어떻게 ->
    부정적 톤 : 너무 느려 ->
4. 데이테셋 검증
  의도별 데이터수 규형(배송:30%, 불량:30%, 문의:30%, 환불:30
5. 파인튜닝준비
```

In [82]:
raw_data = [
    {"question": "배송 언제 오나요?", "answer": "보통 주문 후 2~3일 내 배송됩니다."},
    {"question": "환불 어떻게 하나요?", "answer": "구매 후 7일 이내 반품 요청 시 전액 환불 가능합니다."},
    {"question": "제품이 망가졌어요.", "answer": "고객센터로 연락 주시면 교환 또는 환불 처리해드립니다."},
    {"question": "이거 왜 이래요?", "answer": "자세한 상황을 말씀해 주시면 도와드리겠습니다."},
    {"question": "환불 안 해주면 고소할게요.", "answer": "환불 정책에 따라 처리해드리겠습니다. 고객센터로 연락 부탁드립니다."}
]
policy_doc = """
배송 정책: 모든 주문은 2~3일 내 배송됩니다. 도서산간 지역은 5일 소요.
환불 정책: 구매 후 7일 이내 반품 시 전액 환불. 사용한 제품은 환불 불가.
불량 처리: 불량 제품은 고객센터 '(1234-5678)' 로 연락 후 교환 또는 환불.
"""

In [83]:
# 반자동 라벨링 : OpenAI로 데이터 생성
def generate_faq(indent,policy_doc,num_samples=2):
  prompt = f'''
  당신은 고객지원 데이터셋 생성 전문가입니다. 아래 정책을 참고하여 {indent}
  의도에 맞는 한글 문의-응답 을 존댓말을 사용하고 자연스러운 표현을 사용해서 생성해주세요

  정책:
  {policy_doc}

  출력형식 (JSONL, 각 줄은 하나의 JSON 객체):
  {{"question": "문장1", "answer": "문장2"}}
  {{"question": "문장3", "answer": "문장4"}}
  ...
  총 {num_samples}줄 생성해주세요.
  '''
  try:
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{"role":"user","content":prompt}],
        temperature = 0.7,
        max_tokens = 500,
    )
    temp =  response.choices[0].message.content
    json_lists = []
    for i in temp.split("\n"):
      json_lists.append(json.loads(i))
    return json_lists
  except Exception as e:
    print(f"데이터 생성오류: {e}")
    return []
# 데이터셋  설계
def design_dataset(raw_data,policy_doc):
  # 의도 태그추가
  indent_map = {
      "배송" : "배송",
      "환불" : "환불",
      "제품이 망가" : "불량",
      "이거 왜" : "모호",
      "고소" : "민감"
  }
  datasets = []
  for item in raw_data:
    question = clean_text(item['question'])
    answer = clean_text(item['answer'],is_answer=True)
    for k,v in indent_map.items():
      if k in question:
        indent = v
        break
    else:
      indent = "일반"
    datasets.append({"intent": indent, "question": question, "answer": answer})
  # 반자동 라벨링 : 부족한 의도 보강
  indents = ["배송","환불","불량","모호","민감"]
  for indent in indents:
    if sum(1 for d in datasets if d['intent'] == indent) < 3:
      generated = generate_faq(indent,policy_doc,num_samples=3)
      for item in generated:
        datasets.append({
            'intent' : indent,
            'question' : clean_text(item['question']),
            'answer' : clean_text(item['answer'],is_answer=True),
        })
  return datasets

In [84]:
new_dataset = design_dataset(raw_data,policy_doc)

In [85]:
new_dataset

[{'intent': '배송', 'question': '배송 언제 오나요', 'answer': '보통 주문 후 2~3일 내 배송됩니다'},
 {'intent': '환불',
  'question': '환불 어떻게 하나요',
  'answer': '구매 후 7일 이내 반품 요청 시 전액 환불 가능합니다'},
 {'intent': '불량',
  'question': '제품이 망가졌어요',
  'answer': '고객센터로 연락 주시면 교환 또는 환불 처리해드립니다'},
 {'intent': '모호',
  'question': '이거 왜 이래요',
  'answer': '자세한 상황을 말씀해 주시면 도와드리겠습니다'},
 {'intent': '환불',
  'question': '환불 안 해주면 고소할게요',
  'answer': '환불 정책에 따라 처리해드리겠습니다 고객센터로 연락 부탁드립니다'},
 {'intent': '배송',
  'question': '안녕하세요 제품을 주문한 지 일이 지나도 아직 배송이 안 돼서 문의 드립니다',
  'answer': '안녕하세요 죄송합니다 일반 지역의 경우 2~3일 내에 배송이 완료되지만 도서산간 지역의 경우 5일 소요될 수 있습니다'},
 {'intent': '배송',
  'question': '제품을 사용해본 후에 마음에 들지 않아 반품을 하고 싶은데 환불이 가능한가요',
  'answer': '안녕하세요 네 제품 구매 후 7일 이내에 반품 시 전액 환불이 가능합니다 다만 사용한 제품은 환불이 불가능합니다'},
 {'intent': '배송',
  'question': '제품을 받아보니 불량이 있어서 교환하거나 환불을 받고 싶은데 어떻게 해야 하나요',
  'answer': "안녕하세요 죄송합니다 불량 제품은 고객센터 '(12345678)' 로 연락 후 교환 또는 환불이 가능합니다"},
 {'intent': '환불',
  'question': '상품을 받고 사용해보니 만족스럽지 않아 환불을 신청하고 싶습니다 어떻게 해야 하나

In [86]:
from collections import Counter
Counter([item['intent'] for item in new_dataset])

Counter({'배송': 4, '환불': 5, '불량': 4, '모호': 4, '민감': 3})

In [87]:
# jsonl파일 생성
create_jsonl(new_dataset,'new_dataset_20250509_t.jsonl')

In [88]:
# 파일 업로드
with open("new_dataset_20250509_t.jsonl", "rb") as f:
  file_response = client.files.create(file=f, purpose='fine-tune')
file_id = file_response.id
print(file_id)

file-6aYqj7ddr9hRhYr4NLmd3W


In [89]:
# 모델 파인튜닝
fine_tune_response = client.fine_tuning.jobs.create(
    training_file=file_id,
    model="gpt-3.5-turbo"
)

정책기반 응답 제어
```
  회사정책(환불7일 이내), 법률(불법요청 거부), 윤리(민감한문제처리 방안)
  응답이 정책을 벗어나면 신뢰도나 법적문제가 발생
  fine_tuned 모델이 한글문의에 대해서 정책을 준수하고 민감/모호한 문의에대해 적절히 대응  
```

In [108]:
policy_doc = """
배송 정책: 모든 주문은 2~3일 내 배송됩니다. 도서산간 지역은 5일 소요.
환불 정책: 구매 후 7일 이내 반품 시 전액 환불. 사용한 제품은 환불 불가.
불량 처리: 불량 제품은 고객센터 '(1234-5678)' 로 연락 후 교환 또는 환불.
민감 문의 : 법적 위험, 욕설 시 고겍센터(123-456)로 안내
법률 준수 : 불법 요청(예: 위조품 요구)은 거부
"""
test_query = [
    '배송은 언제 오나요', #일반
    '환불은 어떻게 하나요?', # 일반
    '이거 왜 이래요?', # 모호
    "환불 안 해주면 고소할께요", # 민감
    "위조품 만들어 주세요" # 불법
]

In [109]:
status = client.fine_tuning.jobs.retrieve(fine_tune_response.id)
status.status, status.fine_tuned_model

('succeeded', 'ft:gpt-3.5-turbo-0125:personal::BVAscc6f')

정책기반 응답 제어
```
  회사정책(환불7일 이내), 법률(불법요청 거부), 윤리(민감한문제처리 방안)
  응답이 정책을 벗어나면 신뢰도나 법적문제가 발생
  fine_tuned 모델이 한글문의에 대해서 정책을 준수하고 민감/모호한 문의에대해 적절히 대응  
```

In [125]:
policy_doc = """
배송 정책: 모든 주문은 2~3일 내 배송됩니다. 도서산간 지역은 5일 소요.
환불 정책: 구매 후 7일 이내 반품 시 전액 환불. 사용한 제품은 환불 불가.
불량 처리: 불량 제품은 고객센터 '(1234-5678)' 로 연락 후 교환 또는 환불.
민감 문의 : 법적 위험, 욕설 시 고겍센터(123-456)로 안내
법률 준수 : 불법 요청(예: 위조품 요구)은 거부
모호 정책 : 자세한 내용은 고객센터로 '(1234-5678)' 로 문의해 주세요
"""
test_query = [
    '배송은 언제 오나요', #일반
    '환불은 어떻게 하나요?', # 일반
    '이거 왜 이래요?', # 모호
    "환불 안 해주면 고소할께요", # 민감
    "위조품 만들어 주세요" # 불법
]

In [126]:
# 정책외반 필터링
def post_process_response(response, policy_doc):
  # 금지단어
  banned_words = ['욕설','바보','멍청']
  if any(word in response for word in banned_words):
    return "부적절한 표현이 포함되어 있습니다. 고객센터로 연락 부탁드립니다."
  # 정책 위반 체크
  if "환불" in response and "7일" not in response:
    return "구매 후 7일 이내 반품 요청시 전액 환불 가능합니다."
  if "불법" in response or "위조"  in response:
    return "불법 요청은 처리 할 수 없습니다."
  return response
# openai api 호출
def call_openapi(message, model = 'gpt-3.5-turbo',temperature=0.3,max_tokens=100)  :
  try:
    response = client.chat.completions.create(
        model = model,
        messages = [{"role":"user","content":message}],
        temperature = temperature,
        max_tokens = max_tokens,
    )
    raw_response = response.choices[0].message.content
    tokens = response.usage.total_tokens
    # post processing
    pre_process_response =  post_process_response(raw_response, policy_doc)
    return pre_process_response, tokens
  except Exception as e:
    print(f'API 호출 오류: {e}')
    return None,0

In [127]:
# 테스트
for test in test_query:
  print(f"-----------{test} -------------")
  prompt = [
      {"role":"system","content":"너는 친절한 한글 고객센터 쳇봇이야. 정책 준수, 존댓말 유지"},
      {"role":"user","content":test},
  ]
  response_fine_tuned, tokens = call_openapi(test,status.fine_tuned_model)
  print(f"답변 : {response_fine_tuned} 사용 토큰수 : {tokens}")


-----------배송은 언제 오나요 -------------
답변 : 고객님께서 주문하신 제품은 2~3일 내에 배송됩니다 사용 토큰수 : 49
-----------환불은 어떻게 하나요? -------------
답변 : 환불은 구매 후 7일 이내에 반품하시면 전액 환불이 가능합니다 사용 토큰수 : 53
-----------이거 왜 이래요? -------------
답변 : 무슨 일 있으셨나요 도와드리겠습니다 사용 토큰수 : 35
-----------환불 안 해주면 고소할께요 -------------
답변 : 구매 후 7일 이내 반품 요청시 전액 환불 가능합니다. 사용 토큰수 : 41
-----------위조품 만들어 주세요 -------------
답변 : 불법 요청은 처리 할 수 없습니다. 사용 토큰수 : 50


파인튜닝된 모델 + RAG 결합
```
파인튜닝:모델을 특정 작업(한글고객지원쳇봇)에 최적화, 도메인지식(FAQ응답) 내재화

RAG(Retrieval-Augmented Generation): 외부문서(정책문서 ,FAQ데이터)에서 관련 정보를 검색해 LLM이 이를 기반으로 응답 생성, 최신/정확한 정보를 제공기능

결합 : 파인튜닝된 모델의 도메인 특화 응답(존댓말, 정책준수) + RAG동적 정보검색 --> 정확하고 자연스러운 한글 쳇봇

한글특성: 감정적표현, 모호한문장,정책준수를 RAG로 보완
```

RAG 작동 원리
```
검색단계
  자용자문의("배송언제와?)와 관련된 외부문서 조각(정책문서의 배송정책) 검색
  방법 : 키워드 매칭, 임베딩 기반 유사도검색(sentence-transfomer)
  띄어쓰기나 형태소분석으로 검색 정확도를 높일
생성단계
  검색된 문서조각을 프롬프트에 포함, LLM이 이를 가공해 응답생성
  ex) 검색 : "배송2~3일" ->"주문후 2~3일이내 배송됩니다"
fined_tuning과  결합
  fined-tuning모델은 정책준수,존댓말,도메인스타일 학습
  RAG는 최신/정확한 데이터 제공, 모델은 이를 자연스럽게 표현
```

파인튜닝 + RAG 프로세스
```
문서준비
  외부문서: 정책문서 ,FAQ, 고객지원 로그
검색시스템 설계
  키워드 매칭
  임베딩(유사도계산)
fined_tuned 모델 준비  
프롬프트 설계
  검색된 문서 + 문의 포함
  ex) "정책":배송 "문의": 배송언제? "답변" :
응답생성 및 검증
  LLM이 검색도니 정보 기반으로 응답
  검증 : 정책준수, 정확성(존댓말, 띄어쓰기.등등)
  ```

In [128]:
client = openai.OpenAI(api_key = api_key)

In [129]:
# 간단한 RAG 검색 : 키워드 매칭
def simple_rag(query, document):
  query = clean_text(query)
  # 형태소 분석으로 키워드 추출
  keywords = [word for word, pos in okt.pos(query, norm=True, stem=True) if pos in ['Noun','Verb'] ]
  lines =  document.split('\n')
  # 키워드 매칭
  relevant = [line for line in lines if any( keyword in line for keyword in keywords)]
  return '\n'.join(relevant) or document  # 관련문서가 없으면 전체 반환

def call_openapi(message, model = 'gpt-3.5-turbo',temperature=0.3,max_tokens=100)  :
  try:
    response = client.chat.completions.create(
        model = model,
        messages = message,
        temperature = temperature,
        max_tokens = max_tokens,
    )
    return response.choices[0].message.content, response.usage.total_tokens
  except Exception as e:
    print(f'API 호출 오류: {e}')
    return None,0

In [131]:
# 프롬프트 설계
rag_prompt = '''
당신은 친절한 고객 지원 챗봇입니다. 다음 정책을 참고해서 문의에 정확히 답변하세요, 존댓말 유지.
정책 :
{}
문의:"{}
답변:
'''
for teste in test_query:
  print(f"-----------{teste} -------------")
  # RAG 검색
  retrived_info = simple_rag(teste, policy_doc)
  print(retrived_info)

  # RAG + Fine-Tuning
  prompt = rag_prompt.format(retrived_info,teste)
  message = [
      {"role":"system","content":"너는 친절한 한글 고객센터 쳇봇이야. 정책 준수, 존댓말 유지"},
      {"role":"user","content":prompt},
  ]
  response,tokens =  call_openapi(message,model=status.fine_tuned_model)
  print(f"답변 : {response} 사용 토큰수 : {tokens}")

-----------배송은 언제 오나요 -------------
배송 정책: 모든 주문은 2~3일 내 배송됩니다. 도서산간 지역은 5일 소요.
답변 : 고객님, 주문하신 제품은 2~3일 내에 배송됩니다. 도서산간 지역의 경우 5일 소요될 수 있습니다 사용 토큰수 : 232
-----------환불은 어떻게 하나요? -------------
환불 정책: 구매 후 7일 이내 반품 시 전액 환불. 사용한 제품은 환불 불가.
불량 처리: 불량 제품은 고객센터 '(1234-5678)' 로 연락 후 교환 또는 환불.
답변 : 구매 후 7일 이내 반품 시 전액 환불이 가능하며, 사용한 제품은 환불이 불가능합니다 사용 토큰수 : 280
-----------이거 왜 이래요? -------------
법률 준수 : 불법 요청(예: 위조품 요구)은 거부
답변 : 죄송합니다 고객님 불법 요청은 처리할 수 없습니다 사용 토큰수 : 194
-----------환불 안 해주면 고소할께요 -------------
환불 정책: 구매 후 7일 이내 반품 시 전액 환불. 사용한 제품은 환불 불가.
불량 처리: 불량 제품은 고객센터 '(1234-5678)' 로 연락 후 교환 또는 환불.
민감 문의 : 법적 위험, 욕설 시 고겍센터(123-456)로 안내
답변 : 안녕하세요 죄송합니다 고객님의 요청에 따라 구매 후 7일 이내 반품 시 전액 환불이 가능합니다 사용한 제품은 환불이 불가능하오니 양해 부탁드립니다 사용 토큰수 : 352
-----------위조품 만들어 주세요 -------------
법률 준수 : 불법 요청(예: 위조품 요구)은 거부
답변 : 고객님, 불법 요청에 해당하는 위조품 제작 요청은 거부됩니다. 다른 문의 사항이 있으시면 알려주세요 사용 토큰수 : 222


In [133]:
test_query = [
"주문한 상품이 배송 중이라고 뜨는데, 일주일째 도착을 안 하고 있어요. 어디서 확인할 수 있나요?",
"상품을 받았는데 색상이 주문한 것과 달라요. 교환은 어떻게 진행하나요?",
"결제는 완료됐는데 주문 내역에 아무 것도 안 떠요. 정상적으로 주문이 된 건가요?",
"회원 가입을 했는데 할인 쿠폰이 적용이 안 돼요. 어떻게 해야 하나요?",
"상품 상세 페이지에 사이즈 정보가 없는데, M사이즈의 정확한 치수를 알 수 있을까요?",
]

for teste in test_query:
  print(f"-----------{teste} -------------")
  # RAG 검색
  retrived_info = simple_rag(teste, policy_doc)
  print(retrived_info)

  # RAG + Fine-Tuning
  prompt = rag_prompt.format(retrived_info,teste)
  message = [
      {"role":"system","content":"너는 친절한 한글 고객센터 쳇봇이야. 정책 준수, 존댓말 유지"},
      {"role":"user","content":prompt},
  ]
  response,tokens =  call_openapi(message,model=status.fine_tuned_model)
  print(f"답변 : {response} 사용 토큰수 : {tokens}")

-----------주문한 상품이 배송 중이라고 뜨는데, 일주일째 도착을 안 하고 있어요. 어디서 확인할 수 있나요? -------------
배송 정책: 모든 주문은 2~3일 내 배송됩니다. 도서산간 지역은 5일 소요.
민감 문의 : 법적 위험, 욕설 시 고겍센터(123-456)로 안내
법률 준수 : 불법 요청(예: 위조품 요구)은 거부
답변 : 고객님, 불편을 드려 죄송합니다. 배송이 지연된 경우 고객센터(123-456) 로 연락 주시면 신속히 처리 도와드리겠습니다 사용 토큰수 : 356
-----------상품을 받았는데 색상이 주문한 것과 달라요. 교환은 어떻게 진행하나요? -------------
배송 정책: 모든 주문은 2~3일 내 배송됩니다. 도서산간 지역은 5일 소요.
불량 처리: 불량 제품은 고객센터 '(1234-5678)' 로 연락 후 교환 또는 환불.
답변 : 불량 제품 교환 또는 환불은 고객센터 '(1234-5678)' 로 연락 후 진행 가능합니다 사용 토큰수 : 311
-----------결제는 완료됐는데 주문 내역에 아무 것도 안 떠요. 정상적으로 주문이 된 건가요? -------------
배송 정책: 모든 주문은 2~3일 내 배송됩니다. 도서산간 지역은 5일 소요.
민감 문의 : 법적 위험, 욕설 시 고겍센터(123-456)로 안내
답변 : 고객님, 주문이 정상적으로 완료되면 주문 내역이 마이페이지에 표시됩니다 도서산간 지역의 경우 5일 이내에 배송될 수 있습니다 사용 토큰수 : 307
-----------회원 가입을 했는데 할인 쿠폰이 적용이 안 돼요. 어떻게 해야 하나요? -------------
민감 문의 : 법적 위험, 욕설 시 고겍센터(123-456)로 안내
답변 : 안녕하세요, 먼저 죄송합니다. 고객님의 회원 가입 시 발급된 쿠폰은 '첫 구매 시 사용 가능' 조건이 있습니다. 처음 구매 시 쿠폰을 사용하지 않으셨다면 다음 구매 시 자동으로 적용됩니다. 불편을 드 사용 토큰수 : 304
-----------상품 상세 페이지

품질평가
```
자동 : 키워드 매칭, 유사도 점수  
수동 : 존댓말, 자연스러움 점검
```

In [138]:
policy_doc = """
배송 정책: 모든 주문은 2~3일 내 배송됩니다. 도서산간 지역은 5일 소요.
환불 정책: 구매 후 7일 이내 반품 시 전액 환불. 사용한 제품은 환불 불가.
불량 처리: 불량 제품은 고객센터(1234-5678)로 연락 후 교환 또는 환불.
민감 문의: 법적 위협, 욕설 시 고객센터(1234-5678)로 안내.
"""
test_inquiries = [
    {"inquiry": "배송 언제 오나요?", "golden_answer": "보통 주문 후 2~3일 내 배송됩니다."},
    {"inquiry": "환불 어떻게 하나요?", "golden_answer": "구매 후 7일 이내 반품 요청 시 전액 환불 가능합니다."},
    {"inquiry": "이거 왜 이래요?", "golden_answer": "자세한 상황을 말씀해 주시면 도와드리겠습니다."},
    {"inquiry": "환불 안 해주면 고소할게요.", "golden_answer": "환불 정책에 따라 7일 이내 반품 시 처리 가능합니다. 고객센터로 연락 부탁드립니다."}
]

In [144]:
from sentence_transformers import SentenceTransformer, util

client = openai.OpenAI(api_key=api_key)
# 품질평가 함수
def evaluate_response(response, golden_answer,inquiry):
  # 키워드 매칭
  keywords = {
      "배송" : ["2~3일"],
      "환불" : ["7일"],
      "고소" : ["고객센터"],
      "이거 왜" : ['자세한','도와']
  }
  intent = next((k for k in keywords if k in inquiry), None)
  accuracy = all(kw for kw in keywords.get(intent, []))  if intent else True

  # 유사도 점수 - 다국어 문장의 의미를 파악하는 모델을 이용해서
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings =  model.encode([response,golden_answer], convert_to_tensor=True)
  similarity =  util.cos_sim(embeddings[0], embeddings[1]).item()  # 유사도 점수 (0 ~ 1)

  # 자연스러움(간단하게 "습니다" 포함여부)
  is_natural = any([True for word in ["습니다",'됩니다','합니다'] if word in response])
  return {
    'accuracy':accuracy,
    'similarity':similarity,
    'is_natural':is_natural
  }

In [146]:
# 테스트 및 평가
results = []
issues = []
for item in test_inquiries:
  inquiry = item['inquiry']
  golden_answer = item['golden_answer']
  # RAG + Fine-tuning
  retrieved_info = simple_rag(inquiry, policy_doc)
  prompt = f"""
  당신은 친절한 고객 지원 쳇봇입니다. 다음 정책을 참고해 문의에 정확하게 답변하세요, 존댓말 유지
  정책:
  {retrieved_info}
  문의: "{inquiry}"
  답변:
  """
  messages = [
      {"role":"system","content":"정책 준수, 존댓말 유지"},
      {"role":"user","content":prompt},
      ]
  response,tokens = call_openapi(messages,model=status.fine_tuned_model)
  # print(f"답변 : {response} 사용 토큰수 : {tokens}")

  # 품질 평가
  eval_results = evaluate_response(response,golden_answer,inquiry)
  results.append({
    "inquiry":inquiry,
    "response"  :response,
    "golden_answer" :golden_answer,
    "accuracy" : eval_results['accuracy'],
    "similarity" : eval_results['similarity'],
    "is_natural" : eval_results['is_natural'],
    "tokens" : tokens,
  })
  if not eval_results['accuracy'] or not eval_results['is_natural']:
    issues.append({
      "inquiry":inquiry,
      "response"  :response,
      "accuracy" : eval
    })

In [147]:
import pandas as pd
df= pd.DataFrame(results)
df

,inquiry,response,golden_answer,accuracy,similarity,is_natural,tokens
0,배송 언제 오나요?,"고객님, ""배송 언제 오나요?""라는 문의에 답변드리면, 모든 주문은 2~3일 내에 ...",보통 주문 후 2~3일 내 배송됩니다.,True,0.822503,True,233
1,환불 어떻게 하나요?,구매 후 7일 이내 반품 시 전액 환불이 가능하며 사용한 제품은 환불이 불가능합니다,구매 후 7일 이내 반품 요청 시 전액 환불 가능합니다.,True,0.903919,True,260
2,이거 왜 이래요?,죄송합니다 고객님 어떤 제품에 문제가 있으셨나요,자세한 상황을 말씀해 주시면 도와드리겠습니다.,True,0.558591,True,321
3,환불 안 해주면 고소할게요.,"죄송합니다 고객님, 환불 정책에 따라 사용한 제품은 환불이 불가능합니다",환불 정책에 따라 7일 이내 반품 시 처리 가능합니다. 고객센터로 연락 부탁드립니다.,True,0.677806,True,297
